In [1]:
exec(open('processing_functions.py',encoding='utf-8').read())

from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pandas
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
from scipy.stats import spearmanr as spcorr,pearsonr as pcorr

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
def maxCase(x):
    result = [i for (i,v) in zip(range(-2,3),[y==max(x) for y in x]) if v]
    if(len(result) > 1):
        s = sum(result)
        if(s < -2):
            return -2
        elif(s > 2):
            return 2
        else:
            return s
    else:
        return result[0]
    
def worstCase(x):
    if x['vneg']>0:
        return -2
    elif x['neg']>0:
        return -1
    elif x['neutral']>0:
        return 0
    elif x['pos']>0:
        return 1
    else:
        return 2
    
def bestCase(x):
    if x['vpos']>0:
        return 2
    elif x['pos']>0:
        return 1
    elif x['neutral']>0:
        return 0
    elif x['neg']>0:
        return -1
    else:
        return -2
    
def twoClass(x):
    if x<0:
        return -1
    else:
        return 1
    
sentiments = pandas.read_csv('twitter_sentiment_coreNLP.txt')
sentiments = sentiments.apply(bestCase,axis=1).apply(twoClass)
sentiments.value_counts()

-1    466
 1    437
dtype: int64

In [8]:
tweetFile = open('sallie_mae_tweets.txt','r')
tweetText = tweetFile.read()
tweetText = re.sub(r'\n+(?!Mon|Tue|Wed|Thu|Fri|Sat|Sun)','',string=tweetText)
tweetFile.close()

In [11]:
dates = []
users = []
tweets = []
pattern = re.compile(r'(\w{3} \w{3} \d\d \d\d:\d\d:\d\d \w{3} \d{4}) (@.+?): (.+)')
tweet_breakdown = re.findall(pattern,tweetText)
for i in tweet_breakdown:
    dates.append(i[0])
    users.append(i[1])
    tweets.append(i[2])

In [12]:
tfidf = TfidfVectorizer(lowercase=True, analyzer='word', stop_words='english', tokenizer=LemmaTokenizer,
                        ngram_range=(1, 2), binary=False, norm='l1', use_idf=True, smooth_idf=True)
data = tfidf.fit_transform(tweets)

In [14]:
model = rfc(n_estimators=1000,criterion='entropy',max_features='auto',bootstrap=False,oob_score=False,n_jobs=6,
           random_state=20170202,verbose=1)
model.fit(data,sentiments)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done 1000 out of 1000 | elapsed:    3.2s finished


RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=6,
            oob_score=False, random_state=20170202, verbose=1,
            warm_start=False)

In [15]:
features = tfidf.get_feature_names()
importance = model.feature_importances_
feature_summary = pandas.DataFrame({'name':features,'value':list(importance)})
feature_summary.sort_values('value',ascending=False,inplace=True)

In [16]:
n = 25
ngrams = feature_summary.head(n).index

np.corrcoef(matrix[:,ngrams].todense())

In [19]:
rho = []
pval = []
for a in ngrams:
    r,p = spcorr(data[:,a].toarray().flatten(),sentiments)
    rho.append(r)
    pval.append(p)
    del r,p

In [20]:
corr_summary = pandas.DataFrame({'name':feature_summary.head(n).name,'cor':rho,'pval':pval})
corr_summary.sort_values('cor')

,cor,name,pval
4090,-0.244072,recognize,1.030415e-13
3911,-0.241665,practice,1.820745e-13
4092,-0.239239,recognize cybersecurity,3.211586e-13
837,-0.237693,cybersecurity,4.596063e-13
3071,-0.234329,mae recognize,9.940020e-13
3203,-0.229336,management,3.055423e-12
3204,-0.226806,management practice,5.343309e-12
839,-0.226806,cybersecurity management,5.343309e-12
3975,-0.224272,prologue http,9.293294e-12
3409,-0.183884,navient,2.612879e-08
